# Retriever for relevant Documents

Let's see how well a simple vector store retriever fares in pulling documents based on keywords. We can use this later to build a RAG system

In [2]:
import pandas as pd
import langchain_core as lc

from langchain_community.document_loaders import DataFrameLoader

### Read in data, merge categories and drop uncoded entries

In [3]:
# read data
data = pd.read_csv("CAP_sample_coded.tsv", 
                   sep="\t", 
                     dtype={"_id": str} # _id column as string to avoid scientific notation
                     )

# we remove the categories "wirtschaft" (economy) and "haushalt_finanzen" (finance) in favor of the compound category "wirtschaft_finanzen"
data = data.drop(columns=["wirtschaft", "haushalt_finanzen"])
# rename "wirtschaft_finanzen_merge" to "wirtschaft_finanzen"
data = data.rename(columns={"wirtschaft_finanzen_merge": "wirtschaft_finanzen"})

# pivot data to long format
data_long = data.melt(id_vars=["_id", "intercoder_sample", "coder", "_source.text"], 
                      var_name="label")

data_long = data_long[data_long.value == True] # only keep tweets labeled as a certain category (drop False values)

# remove value and coder column
data_long = data_long.drop(columns=["value", "coder"])

data_long

,_id,intercoder_sample,_source.text,label
802,1623756907532910593,False,Das #Deutschlandticket ist ein richtiger Schri...,verkehr
806,1400799340021616640,False,Das Jobticket ist nun seit einem Monat am Star...,verkehr
825,1044258236663304193,False,Der Antrag zu einem Sozialticket für Bus und B...,verkehr
853,1483489042691940357,False,Die Stadt Leverkusen inkl. Oberbürgermeister u...,verkehr
866,1109875138516959234,False,Diese Kampagne bringt wohl mehr als die üblich...,verkehr
...,...,...,...,...
40770,1360263802294579201,True,Der #Unternehmerlohn für Kulturschaffende und ...,wirtschaft_finanzen
40774,1457645827078434818,True,"Schätzungen gehen davon aus, dass jedes Jahr m...",wirtschaft_finanzen
40778,1522118401937793033,True,Eil: Gericht erklärt #Wirecard-Bilanzen für ni...,wirtschaft_finanzen
40779,1567473763280719872,True,"Fraktionsvize @mueller_sepp stellt fest, für O...",wirtschaft_finanzen


In [4]:
# turn data_long into langchain documents
loader = DataFrameLoader(data_long, page_content_column="_source.text")

loader.load()

[Document(metadata={'_id': '1623756907532910593', 'intercoder_sample': False, 'label': 'verkehr'}, page_content='Das #Deutschlandticket ist ein richtiger Schritt zu einem modernen #ÖPNV. Auch für grenzüberschreitende Mobilität brauchen wir attraktive Angebote und keine Tarif-Hemmnisse. Mobilität darf nicht an der Grenze enden und muss für alle erschwinglich sein! \r\nhttps://t.co/MAsAq3ghPm https://t.co/6HJKoOMnAF'),
 Document(metadata={'_id': '1400799340021616640', 'intercoder_sample': False, 'label': 'verkehr'}, page_content='Das Jobticket ist nun seit einem Monat am Start und immer mehr Arbeitgeber und Arbeitgeberinnen in SH sind dabei!👍🚍🚈 https://t.co/LlPRFh6vAI'),
 Document(metadata={'_id': '1044258236663304193', 'intercoder_sample': False, 'label': 'verkehr'}, page_content='Der Antrag zu einem Sozialticket für Bus und Bahn ist zumindest in einer Lightversion durchgekommen. Wird in der nächsten Verhandlungsrunde des marego-Verbundes behandelt.'),
 Document(metadata={'_id': '148348

### OpenAI embeddings

In [5]:
from langchain_openai import OpenAIEmbeddings

vector_store = lc.vectorstores.InMemoryVectorStore(OpenAIEmbeddings())

vector_store.add_documents(documents=loader.load()) # Tweet IDs are preserved in the metadata. As we have duplicates (multiple labels on the same document), the vector store assigns a new, unqiue ID for each doc 

['b9131ed6-dbc1-497b-b593-ca9c55c0acba',
 'cb97954f-6637-4717-aeb6-1e4940d43d9e',
 'c218ee9e-638c-4027-a730-6d4ba62dbe12',
 '5d0d7e26-3e02-4330-843b-d6dc5f4272f1',
 'ba3becfb-31d9-4a0d-97a2-a5a8237a0cc7',
 '652c13fe-3eab-4f62-b314-cb26d11cc0f0',
 '6eba57cb-c5e2-44d3-826f-186a832bb691',
 '54667123-03de-440b-ad42-f8c8b8aa2a8a',
 '4ce159b0-5ad9-4ff0-9837-536592faeb3d',
 '45ce6d8b-3dfa-4f91-8d69-6b3089c125f0',
 '57cbf31c-e49c-4514-ae7b-61fb56d2b729',
 'c1e615b8-bd47-4ec3-8f5d-1a976425e8c3',
 '9629be87-04d4-4030-8d05-17364aff13fb',
 '2279bdc5-8683-45fe-90a3-ce963d4018f4',
 'a8ea52e0-e0e8-42d5-bbac-77f997506686',
 'f06f0252-8654-4a23-b822-6d4e6fc97f26',
 'b72c1f49-aa02-4061-be68-26b86c913cb3',
 'b9fb8b82-e7fa-499d-b550-1f907fd8536e',
 'e0d721ac-e8ec-4d9a-bc88-b5ab5e3d95b5',
 '9a7406ca-b753-47d7-8606-92c711f881de',
 '295fc493-37c9-432b-b027-caf376ae39c7',
 '3b64ab2d-489d-4691-a467-21e419341711',
 '6fb6b35e-2f47-4648-9b94-3c2c46d0168f',
 '0ebad6de-beef-484c-9af8-a607118b74ae',
 '2b2a4874-73da-

In [6]:
# a simple test

query = "Verkehrspolitik"

vector_store.similarity_search(query, top_k=5)

[Document(id='c19a44b0-4221-497c-a6a6-c788f0051be3', metadata={'_id': '1023670997659131905', 'intercoder_sample': False, 'label': 'aeusseres'}, page_content='"Es ist eindeutig festzustellen, dass die Migrationspolitik einem Fahrplan bzw. einer festgeschriebenen Agenda folgt, in welcher den Nationalstaaten kaum noch eigener Spielraum eröffnet wird." https://t.co/7TaxkcvJFT'),
 Document(id='3cf7b018-2856-4765-be69-0cfb2c8324e3', metadata={'_id': '1023670997659131905', 'intercoder_sample': False, 'label': 'europa'}, page_content='"Es ist eindeutig festzustellen, dass die Migrationspolitik einem Fahrplan bzw. einer festgeschriebenen Agenda folgt, in welcher den Nationalstaaten kaum noch eigener Spielraum eröffnet wird." https://t.co/7TaxkcvJFT'),
 Document(id='5dc4e4d8-a876-4c96-811b-7de1fa9c39da', metadata={'_id': '1023670997659131905', 'intercoder_sample': False, 'label': 'gesellschaft'}, page_content='"Es ist eindeutig festzustellen, dass die Migrationspolitik einem Fahrplan bzw. einer 

In [10]:
# filter by policy field
from langchain_core.documents import Document

def policy_field_filter(doc: Document) -> bool:     # function to filter metadata
    return doc.metadata.get("label") == "verkehr"

vector_store.similarity_search(query, top_k=5, 
                               filter=policy_field_filter) 

[Document(id='c1e615b8-bd47-4ec3-8f5d-1a976425e8c3', metadata={'_id': '1233320231537430528', 'intercoder_sample': True, 'label': 'verkehr'}, page_content='Ich mag es ja kurz &amp; knackig. Diese Übersicht grüner Verkehrspolitik in Berlin ist sehr zu empfehlen. https://t.co/itr8plUfVe'),
 Document(id='b9131ed6-dbc1-497b-b593-ca9c55c0acba', metadata={'_id': '1623756907532910593', 'intercoder_sample': False, 'label': 'verkehr'}, page_content='Das #Deutschlandticket ist ein richtiger Schritt zu einem modernen #ÖPNV. Auch für grenzüberschreitende Mobilität brauchen wir attraktive Angebote und keine Tarif-Hemmnisse. Mobilität darf nicht an der Grenze enden und muss für alle erschwinglich sein! \r\nhttps://t.co/MAsAq3ghPm https://t.co/6HJKoOMnAF'),
 Document(id='c218ee9e-638c-4027-a730-6d4ba62dbe12', metadata={'_id': '1044258236663304193', 'intercoder_sample': False, 'label': 'verkehr'}, page_content='Der Antrag zu einem Sozialticket für Bus und Bahn ist zumindest in einer Lightversion durchg

In [14]:
policy_field_filter

<function __main__.policy_field_filter(doc: langchain_core.documents.base.Document) -> bool>

In [9]:
# another test (what do we get when we input a whole tweet as an example?)

query = data_long["_source.text"].iloc[0]

vector_store.similarity_search(query, top_k=5)

[Document(id='ede524a4-6869-4a78-b138-9775f46669e3', metadata={'_id': '1623756907532910593', 'intercoder_sample': False, 'label': 'verkehr'}, page_content='Das #Deutschlandticket ist ein richtiger Schritt zu einem modernen #ÖPNV. Auch für grenzüberschreitende Mobilität brauchen wir attraktive Angebote und keine Tarif-Hemmnisse. Mobilität darf nicht an der Grenze enden und muss für alle erschwinglich sein! \r\nhttps://t.co/MAsAq3ghPm https://t.co/6HJKoOMnAF'),
 Document(id='0349548a-517c-4764-b448-15135724907b', metadata={'_id': '1400799340021616640', 'intercoder_sample': False, 'label': 'arbeit'}, page_content='Das Jobticket ist nun seit einem Monat am Start und immer mehr Arbeitgeber und Arbeitgeberinnen in SH sind dabei!👍🚍🚈 https://t.co/LlPRFh6vAI'),
 Document(id='5c17d4cf-e65f-40ec-ae6f-4c3a5753b32c', metadata={'_id': '1400799340021616640', 'intercoder_sample': False, 'label': 'verkehr'}, page_content='Das Jobticket ist nun seit einem Monat am Start und immer mehr Arbeitgeber und Ar

In [ ]:
# set up a retriever frontend with MMR search method
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 10, "lambda_mult": 0.5},
)

query = "verkehrspolitik"

retriever.invoke(query) # this actually retrieves reasonable results due to the MMR weighting (only 1 doc does not fit)
# MMR (Maximal Marginal Relevance) aims to diversify search results. the amount of diversification is set via the lambda_mult parameter

[Document(id='c19a44b0-4221-497c-a6a6-c788f0051be3', metadata={'_id': '1023670997659131905', 'intercoder_sample': False, 'label': 'aeusseres'}, page_content='"Es ist eindeutig festzustellen, dass die Migrationspolitik einem Fahrplan bzw. einer festgeschriebenen Agenda folgt, in welcher den Nationalstaaten kaum noch eigener Spielraum eröffnet wird." https://t.co/7TaxkcvJFT'),
 Document(id='0cf66986-a0ca-4271-8661-2701a2fda2f8', metadata={'_id': '1227619494987751430', 'intercoder_sample': False, 'label': 'gesundheit'}, page_content='Die Umwelt- und Verkehrspolitik von Bund, Ländern und Kommunen wirkt! Aktuelle Messdaten zeigen: In deutschen Städten hat sich die Luftqualität 2019 deutlich verbessert. Saubere Luft ist wichtig für Gesundheit und Lebensqualität. Mehr: https://t.co/kbY9uOnmEm @bmu @BMVI https://t.co/yZAniA7T6z'),
 Document(id='daa169bf-9dff-47b4-a5b0-9f89cf39756d', metadata={'_id': '1044258236663304193', 'intercoder_sample': False, 'label': 'soziales'}, page_content='Der Antr

In [16]:
query = data_long["_source.text"].iloc[0]

retriever.invoke(query) # partly OK resuluts, but this is an arbitrary example doc used as query

[Document(id='b9131ed6-dbc1-497b-b593-ca9c55c0acba', metadata={'_id': '1623756907532910593', 'intercoder_sample': False, 'label': 'verkehr'}, page_content='Das #Deutschlandticket ist ein richtiger Schritt zu einem modernen #ÖPNV. Auch für grenzüberschreitende Mobilität brauchen wir attraktive Angebote und keine Tarif-Hemmnisse. Mobilität darf nicht an der Grenze enden und muss für alle erschwinglich sein! \r\nhttps://t.co/MAsAq3ghPm https://t.co/6HJKoOMnAF'),
 Document(id='304ac06b-041e-4e7f-a0ed-7caee67e763c', metadata={'_id': '1400799340021616640', 'intercoder_sample': False, 'label': 'umwelt'}, page_content='Das Jobticket ist nun seit einem Monat am Start und immer mehr Arbeitgeber und Arbeitgeberinnen in SH sind dabei!👍🚍🚈 https://t.co/LlPRFh6vAI'),
 Document(id='3cf7b018-2856-4765-be69-0cfb2c8324e3', metadata={'_id': '1023670997659131905', 'intercoder_sample': False, 'label': 'europa'}, page_content='"Es ist eindeutig festzustellen, dass die Migrationspolitik einem Fahrplan bzw. ei